# Speech Emotion Recognition
이 jupyter notebook에서는 사용자의 음성을 통한(특히 콘서트 상황에서의) 감정 인식을 진행합니다.  
참고한 논문은 다음과 같습니다.

- CHATTERJEE, Rajdeep, et al. Real-time speech emotion analysis for smart home assistants. IEEE Transactions on Consumer Electronics, 2021, 67.1: 68-76.

In [41]:
import os                                            
import math
import numpy as np                     
import matplotlib.pyplot as plt 
import tensorflow as tf

# Live Audio
import librosa                                      
import librosa.display                         
from scipy.fftpack import fft                  
from scipy.io.wavfile import write
from sklearn.decomposition import FastICA


In [4]:
# If you use local machine, run this cell
file_root = "/Users/sebinlee/Desktop/Github/SERPractice"

In [70]:
AUDIO_FOLDER = os.path.join(file_root, "audio-dataset/cheering")

## Preprocessing


In [98]:
def preprocessing(audio_file_path) :
    # Load Audio File
    audio_timeseries, sampling_rate = librosa.load(audio_file_path)

    # Silence Removal
    audio_timeseries, _ = librosa.effects.trim(audio_timeseries)
    audio_timeseries = librosa.power_to_db(audio_timeseries, ref=np.max)

    # Declare variable 
    hop_length = 512
    duration = librosa.get_duration(audio_timeseries, sr=sampling_rate, hop_length=hop_length)
    n_mfcc = math.floor(sampling_rate * duration / hop_length)                                  # Fomula from paper "Real-time speech emotion analysis for smart home assistants"

    # Get mfcc
    mfcc = librosa.feature.mfcc(audio_timeseries, sr=sampling_rate, n_mfcc=n_mfcc, dct_type=3)

    # Run Independent Component Analysis
    ica = FastICA(n_components=3)
    audio_ica = ica.fit_transform(mfcc)


In [99]:
preprocessing(os.path.join(AUDIO_FOLDER, "cheering.wav"))